In [1]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from cell_localization.evaluation import score_coordinates
import numpy as np

def _read_egg_files(src_dir, ext, desc):
    fnames = src_dir.rglob('*' + ext)
    fnames = [x for x in fnames if not x.name.startswith('.')]
    
    data = {}
    for fname in tqdm(fnames, desc = desc):
        df = pd.read_csv(fname)
        bn = fname.name[:-len(ext)]
        data[bn] = df
        
    return data

bn = 'AUG_worm-eggs-adam-masks+Feggs+roi128+hard-neg-5_clf+unet-simple_maxlikelihood_20190808_151948_adam_lr0.000128_wd0.0_batch64'
features_dir = Path.home() / 'workspace/WormData/screenings/pesticides_adam/'
events_dir = Path.home() / 'workspace/WormData/egg_laying/plates/predictions/syngenta' / bn

metadata_file = features_dir / 'Syngenta_Master_Metadata.csv'

files_data = pd.read_csv(metadata_file)
files_data.dropna(subset = ['File_Name'], inplace = True)
files_data['File_Name'] = files_data['File_Name'].map(lambda x : x.rpartition('.')[0])

targets = _read_egg_files(features_dir, '_eggs.csv', 'Reading Ground Truth Files')   
events = _read_egg_files(events_dir, '_eggs_events.csv', 'Reading Events Files')
events_filtered = _read_egg_files(events_dir, '_eggs_events_filtered.csv', 'Reading Events Filtered Files')

Reading Events Filtered Files: 100%|██████████| 1815/1815 [00:08<00:00, 223.03it/s]


In [28]:
metrics_unfilt = np.zeros(3)
metrics_filt = np.zeros(3)
for bn, target_df in tqdm(targets.items()):
    if not bn in events:
        continue
    
    target_df = target_df[target_df['frame_number'] > 0]
    events_df = events[bn]
    events_filt_df = events_filtered[bn]
    
    preds_coords = events_df[['src_x', 'src_y']].values
    target_coords = target_df[['x', 'y']].values
    TP, FP, FN, pred_ind, true_ind = score_coordinates(preds_coords, target_coords, max_dist = 5)
    metrics_unfilt += TP, FP, FN
    
    assert len(events_filt_df) <= len(events_df)
    filt_id = {(x,y) :i for i, (x,y) in enumerate(zip(events_filt_df['x'], events_filt_df['y']))}
    unfilt_id = {(x,y) :i for i, (x,y) in enumerate(zip(events_df['x'], events_df['y']))}
    
    ind_unfilt = [unfilt_id[k] for k in filt_id.keys()]
    
    pred_valid = preds_coords[ind_unfilt]
    TP, FP, FN, pred_ind, true_ind = score_coordinates(pred_valid, target_coords, max_dist = 5)
    metrics_filt += TP, FP, FN
    
    
    
    
    
TP, FP, FN = metrics_unfilt
P = TP/(TP+FP)
R = TP/(TP+FN)
F1 = 2*P*R/(P+R)

print(P, R, F1)

TP, FP, FN = metrics_filt
P = TP/(TP+FP)
R = TP/(TP+FN)
F1 = 2*P*R/(P+R)

print(P, R, F1)


100%|██████████| 1080/1080 [00:02<00:00, 447.52it/s]

0.7964744624523794 0.952656780970034 0.8675927554070687
0.9326076320939335 0.5888940376892184 0.7219276652149215


In [26]:
ind_unfilt = [unfilt_id[k] for k in filt_id.keys()]
